In [4]:
!nvidia-smi

Mon Dec 25 03:53:24 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [5]:
import os
HOME = os.getcwd()
print(HOME)

/content


## Install Grounding DINO 🦕

In [6]:
%cd {HOME}
!git clone https://github.com/IDEA-Research/GroundingDINO.git
%cd {HOME}/GroundingDINO
!pip install -q -e .
!pip install -q roboflow

/content
Cloning into 'GroundingDINO'...
remote: Enumerating objects: 418, done.
remote: Counting objects: 100% (181/181), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 418 (delta 138), reused 126 (delta 122), pack-reused 237
Receiving objects: 100% (418/418), 12.85 MiB | 19.70 MiB/s, done.
Resolving deltas: 100% (215/215), done.
/content/GroundingDINO
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.7/254.7 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.5/77.5 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.0/69.0 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.3/158.3 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━

In [7]:
import os

CONFIG_PATH = os.path.join(HOME, "GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py")
print(CONFIG_PATH, "; exist:", os.path.isfile(CONFIG_PATH))

/content/GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py ; exist: True


## Download Grounding DINO Weights 🏋️

In [8]:
%cd {HOME}
!mkdir {HOME}/weights
%cd {HOME}/weights

!wget -q https://github.com/IDEA-Research/GroundingDINO/releases/download/v0.1.0-alpha/groundingdino_swint_ogc.pth

/content
/content/weights


In [9]:
import os

WEIGHTS_NAME = "groundingdino_swint_ogc.pth"
WEIGHTS_PATH = os.path.join(HOME, "weights", WEIGHTS_NAME)
print(WEIGHTS_PATH, "; exist:", os.path.isfile(WEIGHTS_PATH))

/content/weights/groundingdino_swint_ogc.pth ; exist: True


## Load Grounding DINO Model

In [11]:
%cd {HOME}/GroundingDINO

from groundingdino.util.inference import load_model, load_image, predict, annotate

model = load_model(CONFIG_PATH, WEIGHTS_PATH)

/content/GroundingDINO


/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


final text_encoder_type: bert-base-uncased


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [2]:
# Python code to install necessary libraries for the PDF extraction and conversion process
!pip install rarfile PyMuPDF Pillow


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 49.9 MB/s eta 0:00:00


In [3]:
import os
import rarfile
import fitz  # PyMuPDF
from PIL import Image

def extract_rar(rar_path, extract_path):
    """
    Extracts a RAR file to a specified directory.
    """
    with rarfile.RarFile(rar_path) as rf:
        rf.extractall(path=extract_path)
    print(f"Extracted RAR contents to {extract_path}")

def convert_pdf_to_images(pdf_path, output_dir, dpi=600):
    """
    Converts each page of a PDF to an image and saves it in a specified directory.
    """
    doc = fitz.open(pdf_path)
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)  # load page
        pix = page.get_pixmap(dpi=dpi)  # render page to an image
        img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
        img.save(os.path.join(output_dir, f"page_{page_num + 1}.png"))
    doc.close()


def process_rar_to_images(rar_path, output_base_path):
    """
    Extracts RAR and converts all contained PDFs to images, each in its own directory.
    """
    # Extract RAR
    extract_path = "/content/drive/MyDrive/pdf_image/extracted/100_PDF"
    extract_rar(rar_path, extract_path)

    # Process each PDF
    for item in os.listdir(extract_path):
        if item.lower().endswith('.pdf'):
            pdf_path = os.path.join(extract_path, item)
            pdf_name = os.path.splitext(item)[0]
            output_dir = os.path.join(output_base_path, pdf_name)
            os.makedirs(output_dir, exist_ok=True)
            convert_pdf_to_images(pdf_path, output_dir)
            print(f"Converted {item} to images in {output_dir}")

    print("Completed processing all PDFs in the RAR file.")

# Example usage
rar_path = "/content/drive/MyDrive/pdf.rar" # Replace with your RAR file path
output_base_path = "/content/drive/MyDrive/pdf_image_4"  # Base path for output, change if needed

# Uncomment the line below to run the function
process_rar_to_images(rar_path, output_base_path)


Extracted RAR contents to /content/drive/MyDrive/pdf_image/extracted/100_PDF
Converted elsevier_354c8aacec9691b4a9c1334bccc7818d224b2971.pdf to images in /content/drive/MyDrive/pdf_image_4/elsevier_354c8aacec9691b4a9c1334bccc7818d224b2971
Converted elsevier_0d8e1fc979302d36e57072865329cbb30781292d.pdf to images in /content/drive/MyDrive/pdf_image_4/elsevier_0d8e1fc979302d36e57072865329cbb30781292d
Converted elsevier_060c68226fbc74c162409aeec9ed07224e9a1ca9.pdf to images in /content/drive/MyDrive/pdf_image_4/elsevier_060c68226fbc74c162409aeec9ed07224e9a1ca9
Converted elsevier_05cbcb9ef5629bc25e84df43572f9d1eddb9a35f.pdf to images in /content/drive/MyDrive/pdf_image_4/elsevier_05cbcb9ef5629bc25e84df43572f9d1eddb9a35f
Converted elsevier_36b9d6d65905af4c409014d17de90ed316495a9f.pdf to images in /content/drive/MyDrive/pdf_image_4/elsevier_36b9d6d65905af4c409014d17de90ed316495a9f
Converted elsevier_219a105beec6e4543076e6e08395200ece078488.pdf to images in /content/drive/MyDrive/pdf_image_4/e

In [ ]:
!apt-get install unrar
import os

# RAR文件路径
rar_file = '/content/drive/MyDrive/pdf.rar'

# 解压目标文件夹
destination_folder = '/content/GroundingDINO/pdfimg'

# 创建目标文件夹（如果不存在）
if not os.path.exists(destination_folder):
    os.makedirs(destination_folder)

# 解压RAR文件
os.system(f'unrar x {rar_file} {destination_folder}')


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
unrar is already the newest version (1:6.1.5-1).
0 upgraded, 0 newly installed, 0 to remove and 15 not upgraded.


0

In [13]:
import zipfile
import os

# 指定要打包的文件夹和输出的ZIP文件名
folder_path = '/content/GroundingDINO/pdfimg/100_PDF/annotated_images_6'
zip_file = 'pdf_image_dino_highres.zip'

# 创建一个ZipFile对象
with zipfile.ZipFile(zip_file, 'w') as zipf:
    # 遍历文件夹中的所有文件和子文件夹
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            # 将每个文件添加到zip文件中
            zipf.write(os.path.join(root, file))

print("打包完成")


打包完成


In [12]:
import torchvision.transforms.functional as TF
import os
import numpy as np
import torch
import matplotlib.pyplot as plt
from matplotlib.image import imread
from torchvision.ops import box_convert
TEXT_PROMPT = "graph"
BOX_TRESHOLD = 0.35
TEXT_TRESHOLD = 0.25
PDF_IMG_FOLDER = "/content/drive/MyDrive/pdf_image_4"
OUTPUT_FOLDER = "/content/GroundingDINO/pdfimg/100_PDF/annotated_images_6"

def get_files_in_directory(directory):
    files = []
    for filename in os.listdir(directory):
        filepath = os.path.join(directory, filename)
        if os.path.isfile(filepath):
            files.append(filepath)
    return files

def process_pdf(pdf_img_dir):
    base_name = os.path.splitext(os.path.basename(pdf_img_dir))[0]
    output_images_folder = os.path.join(OUTPUT_FOLDER, base_name)
    os.makedirs(output_images_folder, exist_ok=True)
    pages = get_files_in_directory(pdf_img_dir)
    for i, page in enumerate(pages):
        image_path = os.path.join(output_images_folder, f"{i}.jpg")
        image_source, image = load_image(page)
        boxes, logits, phrases = predict(
              model=model,
              image=image,
              caption=TEXT_PROMPT,
              box_threshold=BOX_TRESHOLD,
              text_threshold=TEXT_TRESHOLD
          )
        annotated_frame = annotate(image_source=image_source, boxes=boxes, logits=logits, phrases=phrases)
        boxes = box_convert(boxes=boxes, in_fmt="cxcywh", out_fmt="xyxy")
        crop_and_save_images(page, boxes, output_images_folder, i)


def crop_and_save_images(image_path, boxes, output_path, i):
    # 读取原始图像
    img = imread(image_path)
    img_height, img_width = img.shape[:2]

    # 确保输出路径存在
    if not os.path.exists(output_path):
        os.makedirs(output_path)

    # 遍历boxes数组
    for j, box in enumerate(boxes):
        # 计算裁剪坐标
        left = int(box[0].item() * img_width)
        top = int(box[1].item() * img_height)
        right = int(box[2].item() * img_width)
        bottom = int(box[3].item() * img_height)

        if (box[0].item() <= 0.1 and box[1].item() <= 0.1 and box[2].item() >= 0.9 and box[3].item() >= 0.9):
          continue

        # 裁剪图像
        cropped_img = img[top:bottom, left:right]

        # 保存裁剪后的图像
        try:
          plt.imsave(os.path.join(output_path, f"{i}_{j}.jpg"), cropped_img)
        except Exception as e:
          print(e)



for dir in os.listdir(PDF_IMG_FOLDER):
    process_pdf(os.path.join(PDF_IMG_FOLDER, dir))

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:907: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:907: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/usr

In [ ]:
import os
import cv2
from pdf2image import convert_from_path
from GroundingDINO.groundingdino.util.inference import load_model, load_image, predict, annotate, Model

TRANSFORMERS_OFFLINE=1
CONFIG_PATH = "GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py"
CHECKPOINT_PATH = "./GroundingDINO/groundingdino_swint_ogc.pth"
DEVICE = "cuda"
TEXT_PROMPT = "Illustration picture in an academic paper"
BOX_TRESHOLD = 0.35
TEXT_TRESHOLD = 0.25
PDF_IMG_FOLDER = "./100_PDF/images"
OUTPUT_FOLDER = "./100_PDF/annotated_images"
model = load_model(CONFIG_PATH, CHECKPOINT_PATH)

def get_files_in_directory(directory):
    files = []
    for filename in os.listdir(directory):
        filepath = os.path.join(directory, filename)
        if os.path.isfile(filepath):
            files.append(filepath)
    return files

def process_pdf(pdf_img_dir):
    base_name = os.path.splitext(os.path.basename(pdf_img_dir))[0]
    output_images_folder = os.path.join(OUTPUT_FOLDER, base_name)
    os.makedirs(output_images_folder, exist_ok=True)
    pages = get_files_in_directory(pdf_img_dir)
    for i, page in enumerate(pages):
        image_path = os.path.join(output_images_folder, f"{i}.jpg")
        image_source, image = load_image(page)
        boxes, logits, phrases = predict(
            model=model, image=image, caption=TEXT_PROMPT,
            box_threshold=BOX_TRESHOLD, text_threshold=TEXT_TRESHOLD,
            device=DEVICE
        )
        annotated_frame = annotate(image_source=image_source, boxes=boxes, logits=logits, phrases=phrases)
        cv2.imwrite(image_path, annotated_frame)

for dir in os.listdir(PDF_IMG_FOLDER):
    process_pdf(os.path.join(PDF_IMG_FOLDER, dir))
